

Ah, tôi xem lại code và thấy pipeline đang lấy data từ Hugging Face dataset:

```python
# Current data source
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
```

Đây là dataset về 7 kỳ quan thế giới. Thay vì lấy data từ dataset này, chúng ta cần truy vấn trực tiếp từ Qdrant collection `trung_luat_DB` của bạn. Sửa lại như sau:

```python:pipelines_OpenWebUI_aPartOf_RAG6/pipelines/haystack_pipeline.py
    async def on_startup(self):
        os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

        # ... keep existing imports ...

        # Thay đổi phần lấy data
        from haystack import Document
        
        # Lấy documents từ Qdrant
        search_results = self.qdrant_client.scroll(
            collection_name=self.collection_name,
            limit=100  # Có thể điều chỉnh số lượng
        )[0]  # [0] là danh sách points, [1] là next_page_offset

        # Chuyển đổi Qdrant points thành Haystack Documents
        docs = [
            Document(
                content=point.payload.get("content", ""),
                meta=point.payload.get("metadata", {})
            ) 
            for point in search_results
        ]

        # Phần còn lại giữ nguyên...
```

Như vậy pipeline sẽ:
1. Lấy data từ Qdrant collection của bạn thay vì dataset seven-wonders
2. Chuyển đổi data từ Qdrant format sang Haystack Document format
3. Tiếp tục xử lý như pipeline cũ với embedder, retriever, v.v.




Ah, tôi thấy cấu trúc dữ liệu trong Qdrant của bạn có:
1. `page_content`: chứa nội dung văn bản
2. `metadata`: chứa source file

Vậy ta cần điều chỉnh mapping trong file `qdant_pipeline.py` như sau:

```python:pipelines_OpenWebUI_aPartOf_RAG6/pipelines/qdant_pipeline.py
# Trong hàm on_startup()

# Kiểm tra và chuyển đổi dữ liệu từ Qdrant sang Haystack Document
def convert_qdrant_to_document(point):
    return Document(
        content=point.payload.get("page_content", ""),  # map từ page_content
        meta={
            "source": point.payload.get("metadata", {}).get("source", ""),  # map từ metadata.source
            "id": point.id
        }
    )

# Lấy documents từ Qdrant
search_results = document_store.get_all_documents()
docs = [convert_qdrant_to_document(point) for point in search_results]

# Phần còn lại giữ nguyên...
```

Cấu trúc này sẽ map:
- `page_content` -> `Document.content`
- `metadata.source` -> `Document.meta["source"]`
- `point.id` -> `Document.meta["id"]`

Như vậy sẽ tương thích với cấu trúc Document mà Haystack pipeline đang sử dụng.


# Chuyển kiểu: 

1. import trực tiếp vào docker 
```
docker exec -it pipelines bash
pip install qdrant-client
```